
### 🏷️ **Credits & License**

* 🔗 [VibeVoice GitHub Repository](https://github.com/microsoft/VibeVoice)
* 🤗 [VibeVoice-Realtime-0.5B on Hugging Face](https://huggingface.co/microsoft/VibeVoice-Realtime-0.5B)
* 📄 **License**: Provided under the [MIT License](https://github.com/microsoft/VibeVoice/blob/main/LICENSE)


In [1]:
#@title Install VibeVoice-Realtime-0.5B
%cd /content/
#cloudflare and model download setup
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared && chmod +x cloudflared
!wget https://raw.githubusercontent.com/NeuralFalconYT/Useful-Function/refs/heads/main/hf_downloader.py

#Git clone and Installe dependencies
!git clone https://github.com/microsoft/VibeVoice.git
%cd /content/VibeVoice
!wget https://raw.githubusercontent.com/NeuralFalconYT/VibeVoice-Realtime-0.5B-colab/refs/heads/main/app.py
!uv pip --quiet install --system -e /content/VibeVoice

# !pip install gradio
# !pip install torch
# !pip install accelerate==1.6.0
# !pip install transformers==4.51.3
# !pip install llvmlite>=0.40.0
# !pip install numba>=0.57.0
# !pip install diffusers==0.35.2
# !pip install numpy>=2.0.2
# !pip install scipy>=1.16.3
# !pip install librosa>=0.11.0
# !pip install ml-collections

from IPython.display import clear_output
clear_output()


In [ ]:
#@title Official Flask app
%cd /content/
print("✅ Downloaded model: microsoft/VibeVoice-Realtime-0.5B")
try:
  from huggingface_hub import snapshot_download
  snapshot_download("microsoft/VibeVoice-Realtime-0.5B", local_dir="/content/models/VibeVoice-Realtime-0.5B")
except:
  from hf_downloader import download_model
  download_model("microsoft/VibeVoice-Realtime-0.5B","/content/models/VibeVoice-Realtime-0.5B", redownload= False)



import subprocess, re, time, threading

srv = subprocess.Popen(
    "python /content/VibeVoice/demo/vibevoice_realtime_demo.py --model_path /content/models/VibeVoice-Realtime-0.5B --port 8000",
    shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True,
)
cf = subprocess.Popen(
    "./cloudflared tunnel --url http://localhost:8000 --no-autoupdate",
    shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True,
)

public_url = None
server_ready = False
url_pattern  = re.compile(r"(https://[a-z0-9-]+\.trycloudflare\.com)")

def read_srv():
    global server_ready
    for ln in srv.stdout:
        print(ln.strip())
        if "Uvicorn running on" in ln:
            server_ready = True

def read_cf():
    global public_url
    for ln in cf.stdout:
        m = url_pattern.search(ln)
        if m:
            public_url = m.group(1)
            break

threading.Thread(target=read_srv, daemon=True).start()
threading.Thread(target=read_cf,  daemon=True).start()

from IPython.display import clear_output
clear_output()

while True:
    if server_ready and public_url:
        print(f"✅ Public URL: {public_url}\n");
        public_url = None
    time.sleep(0.25)

In [ ]:
#@title Gradio App
%cd /content/VibeVoice
!python app.py --share --debug